In [1]:
import cv2
import numpy as np
img=cv2.imread("image3.tif",0)
height=img.shape[0]
width=img.shape[1]
print('The shape of original image:',img.shape)
cv2.imshow('Original Image',img)

# -----step1: Smoothing the Original Image with a n*n Gaussian filter-----------------------------------------
sigma=1
n=7
img_smoothed=cv2.GaussianBlur(img,(n,n),sigma)
cv2.imshow('Smoothed Image',img_smoothed)

#-------step2: Finding Intensity Gradient of the Smoothed Image------------------------------------------------
# 2-1: Calculate the 1st-gradient in x and y direction using Sobel filter
Gx = cv2.Sobel(img,cv2.CV_16S,1,0) # get the 1st gradient in X direction 
absX = cv2.convertScaleAbs(Gx)   # 转回uint8
Gy=cv2.Sobel(img,cv2.CV_16S,0,1) # get the 1st gradient in Y direction
absY = cv2.convertScaleAbs(Gy) 

cv2.imshow('Gradient Image in X_direction',absX)
cv2.imshow('Gradient Image in Y_direction',absY)
# 2-2: Calculte the Maginitude and Angle
Mag=np.hypot(absX, absY)
Mag = Mag / Mag.max() * 255
Mag=np.uint8(Mag)

Angle = np.arctan2(absY,absX)
cv2.imshow('Gradient Image ',Mag)
#-------step3: Non-maximum suppression,to remove false responses to to edge detection.------------------------
Img_suppressed= np.copy(Mag)
Theta = (Angle/np.pi) * 180 # radian to degree conversion
Theta[Theta<0] += 180 # scale in range [0,180] degree
# if the value if Mag(x,y) is less than at least one of its two neighbors let Img_suppressed be 0
for i in range(1,height-1): 
    for j in range(1,width-1):
        if (Theta[i,j]<=22.5 or Theta[i,j]>157.5): # horizontal orientation
            if(Mag[i,j]<=Mag[i,j-1]) or (Mag[i,j]<=Mag[i,j+1]): Img_suppressed[i,j]=0
        if (Theta[i,j]>22.5 and Theta[i,j]<=67.5): # -45 degree orientation
            if(Mag[i,j]<=Mag[i-1,j+1]) or (Mag[i,j]<=Mag[i+1,j-1]): Img_suppressed[i,j]=0
        if (Theta[i,j]>67.5 and Theta[i,j]<=112.5): # Vertical orientation
            if(Mag[i,j]<=Mag[i-1,j]) or (Mag[i,j]<=Mag[i+1,j]): Img_suppressed[i,j]=0
        if (Theta[i,j]>112.5 and Theta[i,j]<=157.5): # + 45 degree orientation
            if(Mag[i,j]<=Mag[i-1,j-1]) and (Mag[i,j]<=Mag[i+1,j+1]): Img_suppressed[i,j]=0
cv2.imshow('Suppressed Gradient Image ',Img_suppressed)
#-------step4: Double Threshold-------------------------------------------------------------------------------
lowthreshold=90
highthreshold=150
#-------step5: Edge Tracking by Hysteresis.----------------------------------------------------------------
edge_final = np.zeros((height,width))
for i in range(1,height-1):
    for j in range(1,width-1):
        if(Img_suppressed[i,j] < lowthreshold):
            edge_final[i,j]=0
        elif (Img_suppressed[i,j]>highthreshold):
            edge_final[i,j]=255 # marked as valid edge
        # check 8-connected neighboring pixels
        elif(Img_suppressed[i+1,j]>highthreshold or Img_suppressed[i-1,j]>highthreshold or \
             Img_suppressed[i,j-1]>highthreshold or Img_suppressed[i,j+1]>highthreshold or \
             Img_suppressed[i+1,j+1]>highthreshold or Img_suppressed[i-1,j-1]>highthreshold or \
             Img_suppressed[i+1,j-1]>highthreshold or Img_suppressed[i-1,j+1]>highthreshold ):
            edge_final[i,j]=255 # marked as valid edge
            
edge_final = np.uint8(edge_final)               
cv2.imshow('The Final Edge Image ',edge_final)            

k = cv2.waitKey(0)
if k == 27:
    cv2.destroyAllWindows()

The shape of original image: (834, 1114)


In [2]:
type(Theta)

numpy.ndarray

In [3]:
Theta.dtype

dtype('float16')

In [7]:
Theta

array([[ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [90.  , 82.9 , 80.  , ..., 45.  , 45.  , 90.  ],
       [90.  , 45.  , 90.  , ..., 90.  , 71.56, 90.  ],
       ...,
       [90.  , 65.  , 21.05, ..., 22.73, 26.  , 90.  ],
       [90.  , 80.1 , 88.  , ..., 31.8 , 49.44, 90.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ]], dtype=float16)

In [11]:
Angle.max()

1.57